<a href="https://colab.research.google.com/github/dinn16MU/402GroupProject/blob/main/DS402_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading modules and funcions

In [4]:
import datascience
import numpy as np
import pandas as pd

from datascience import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import metrics

Loading data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
path= "/content/drive/My Drive/DS 402/New_Data.csv"
data = pd.read_csv(path)
data.head(5)

,Age,Course,Gender,CGPA,Stress_Level,Depression_Score,Anxiety_Score,Sleep_Quality,Physical_Activity,Diet_Quality,Social_Support,Relationship_Status,Substance_Use,Counseling_Service_Use,Family_History,Chronic_Illness,Financial_Stress,Extracurricular_Involvement,Semester_Credit_Load,Residence_Type
0,25,Others,Male,3.56,3,3,2,Good,Moderate,Good,Moderate,Married,Never,Never,No,No,2,Moderate,17,On-Campus
1,24,Engineering,Female,2.44,0,3,0,Average,Low,Average,Low,Single,Occasionally,Occasionally,No,No,3,Low,27,On-Campus
2,19,Business,Female,3.74,4,0,3,Good,Low,Average,Moderate,In a Relationship,Never,Occasionally,No,No,4,High,15,On-Campus
3,19,Computer Science,Male,3.65,2,1,0,Average,Low,Average,Moderate,Single,NaN,Never,No,No,4,Moderate,20,Off-Campus
4,18,Business,Male,3.40,3,3,4,Good,Low,Average,High,Married,Never,Never,No,Yes,0,High,23,On-Campus


data preprocessing

In [13]:
data.isna().sum()

,0
Age,0
Course,0
Gender,0
CGPA,12
Stress_Level,0
Depression_Score,0
Anxiety_Score,0
Sleep_Quality,0
Physical_Activity,0
Diet_Quality,0


In [15]:
data = data.dropna()

In [16]:
data.shape

(6995, 20)

In [25]:
data['risk'] = ((data['Depression_Score'] > 3) | (data['Anxiety_Score'] > 3)).astype(int)

In [26]:
label_counts = data['risk'].value_counts()
print("Number of observations for each label:\n", label_counts)

Number of observations for each label:
 risk
0    3926
1    3069
Name: count, dtype: int64
